In [7]:
import numpy as np
import pandas as pd
from pathlib import Path
import imageio
import matplotlib.pyplot as plt
from math import exp

In [3]:
# given heat_map, half_life, and timestamp calculate nexframe heat value
def calculate_frame_heat(heat_map, half_life, dt):
    return heat_map * 2 ** (-dt / half_life)



In [4]:
# Hight fall off over time

def calculate_falloff(P_0, z, H):
    try:
        # Get the hight when z is a single value
        return P_0 * exp(-z / H)
    except TypeError:
        # Get hieght when z is more than one value as np array
        return P_0 * np.exp(-z / H)

In [ ]:
import imageio.v2 as imageio
TIMESTAMP_MAX = 300_589_892


def generate(
    infile,
    colorpath,
    datapath,
    start_ms=0,
    timescale=1000,
    frames=600,
    fps=60,
    heat_half_life=10 * 60 * 1000,
    scale_height=0.3,
):
    """Generate the heat and color frames for the given file."""

    df = pd.read_parquet(infile)

    # The time gap between frames.
    dt = round(timescale * 1000 / fps)

    # Converts earlier hex to and rgb tuple
    indexed_rgb = (
        (0, 0, 0),
        (0, 117, 111),
        (0, 158, 170),
        (0, 163, 104),
        (0, 204, 120),
        (0, 204, 192),
        (36, 80, 164),
        (54, 144, 234),
        (73, 58, 193),
        (81, 82, 82),
        (81, 233, 244),
        (106, 92, 255),
        (109, 0, 26),
        (109, 72, 47),
        (126, 237, 86),
        (129, 30, 159),
        (137, 141, 144),
        (148, 179, 255),
        (156, 105, 38),
        (180, 74, 192),
        (190, 0, 57),
        (212, 215, 217),
        (222, 16, 127),
        (228, 171, 255),
        (255, 56, 129),
        (255, 69, 0),
        (255, 153, 170),
        (255, 168, 0),
        (255, 180, 112),
        (255, 214, 53),
        (255, 248, 184),
        (255, 255, 255),
    )

    # Generate the canvases to hold the running values
    img_color = np.full((2000, 2000, 3), 255, dtype=np.uint8)
    img_heat = np.zeros((2000, 2000), dtype=np.float32)

    # Create an iterator that yields the rows of the dataset in order.
    px_iterator = df.itertuples()

    # Get the first pixel.
    px = next(px_iterator)

    # We won't bother calculating the heat for pixels that will be more than ten
    # half lives old by the time we reach the first frame. At ten half lives,
    # a pixel will be 1/1024th of it's initial height. We also make sure
    # the the buffer time is a multiple of the frame gap time (dt), so that the start
    # time is precise to the millisecond.
    heat_map_buffer_time = dt * (10 * heat_half_life // dt)
    calculation_start_time = max(0, start_ms - heat_map_buffer_time)

    # Iterate through the frames.
    frame_no = 0
    for ms in range(calculation_start_time, TIMESTAMP_MAX, dt):
        # Stop after the last frame.
        if frame_no >= frames:
            break

        # Draw pixels where timestamp <= ms
        while px.timestamp <= ms:
            # Draw the pixel's color to the color canvas.
            img_color[px.y, px.x] = indexed_rgb[px.pixel_color]

            # Make the added height for new pixels
            # decay exponentially with initial height.
            H_0 = 0.1
            img_heat[px.y, px.x] += calculate_falloff(
                H_0,  # Height increase for a pixel with starting height of 0
                img_heat[px.y, px.x],  # Initial height of this pixel
                scale_height,
            )

            try:
                # Get the next pixel.
                px = next(px_iterator)
            except StopIteration:
                # Break out of the loop if we've reached the end of the dataset.
                break

        # After all of the pixels less than the frame's timestamp have been drawn,
        # save the color and birthtime canvases as frames.
        img_heat = calculate_frame_heat(img_heat, heat_half_life, dt)

        if ms < start_ms:
            # Don't save the frame if it's before the start time.
            continue

        zero_arr = np.zeros((2000, 2000), dtype=np.float32)
        img_data = np.dstack((zero_arr, img_heat, zero_arr))

        # Save the frames.
        imageio.imwrite(
            colorpath / f"frame-{str(frame_no).zfill(4)}.png",
            img_color,
            optimize=True,
        )

        imageio.imwrite(
            datapath / f"frame-{str(frame_no).zfill(4)}.exr",
            img_data,
        )

        frame_no += 1


# Download the FreeImage backend library if it doesn't exist.
imageio.plugins.freeimage.download()

generate(
    Path("path to the .parquet file"),
    Path("path to your ..//frame_colors folder"),
    Path("path to your ..//frame_heats folder"),
    start_ms=99645924 + 60 * 60 * 1000,  # one hour after first expansion
    # start_ms=195583041,  # second expansion
    # start_ms=295409870,  # last non-white pixel placed
    frames=1,  # Only save the first frame.
    timescale=(5 * 60 * 60 / 20),  # five hours => 20 seconds
    heat_half_life=(30 * 60 * 1000),  # 30 minute half life.
)

# Show the first color frame.
img_color = imageio.imread("../data/frames_color/frame-0000.png")
plt.imshow(img_color)